In [ ]:
# The packages required for this project are listed below

import pandas as pd # Dataset preparation and "pythonisation"
import matplotlib.pyplot as plt # Dataset visualisation
import numpy as np # Linear algebra utilities

The dataset we'll be analysing is the [Kaggle Netflix Movies and TV Shows](https://www.kaggle.com/datasets/shivamb/netflix-shows) dataset. The local copy used in this analysis is accurate as of August 26, 2022.

In [ ]:
netflix = pd.read_csv("data/netflix_titles.csv")
netflix.columns

### Dataset Preparation 

The ```title```, ```director```, ```cast```, ```show_id```, ```rating``` and ```description``` columns will not be needed for a numerical analysis and will be deleted.

In [ ]:
netflix.drop(["show_id", "title", "director", "cast", "description", "rating"], axis=1, inplace=True)
netflix

In addition, we will only be analysing data from 2000 onwards, hence titles that have ```release_year```s before that must be deleted.

In [ ]:
netflix = netflix.drop(netflix[netflix["release_year"] < 2000].index)

Now that our dataset has been slimmed down, we will check for any null values. Entries with such values will be investigated in order to make sure that the error is not systematic. 

In [ ]:
def checkForNullEntries():
	return netflix.isnull().any()

checkForNullEntries()

We can see that the country, date_added and duration columns have been identified to have null values. It will help to define a function that can filter entries by the presence of null values in a defined column.

In [ ]:
def filterIncompleteEntries(col):
	entries = netflix[netflix[col].isnull()]
	return entries

The most difficult column to filter will be ```country```, so we'll start with that.

In [ ]:
country_incomplete_entries = filterIncompleteEntries("country")
country_incomplete_entries["country"]

The only reasonable aspect of an entry that could lead to a systemic error would be the listed_in column (which is also linked to the type column), hence examining what values are set for this column for these entries should reveal any systemic trends.

In [ ]:
set(country_incomplete_entries["listed_in"])

As expected, some categories stand out. We notice that some shows have been labelled as "British TV Shows" and some others have been labelled as "Korean TV Shows". The countries that these shows/movies can be attributed to is clear. One other intriguing category was "Spanish TV Shows", though it is unclear if this refers to the language or the country. Aside from these, there are no other categories that explicity stand out, hence these entries must be deleted.

In [ ]:
british_mask = netflix["listed_in"].apply(lambda categories: "British TV Shows" in categories)
netflix.loc[british_mask, "country"] = "United Kingdom"

Thus all entries listed in "British TV Shows" have their country attributes correctly marked as "United Kingdom".

In [ ]:
netflix[british_mask]

We do the same for Korean TV shows.

In [ ]:
korean_mask = netflix["listed_in"].apply(lambda categories: "Korean TV Shows" in categories)
netflix.loc[korean_mask, "country"] = "South Korea"

Now we can check whether the data has been correctly modified.

In [ ]:
netflix[korean_mask]

All that remains now is to delete the extra rows.

In [ ]:
netflix = netflix.drop(netflix.index[netflix["country"].isnull()])
checkForNullEntries()

As expected, none of the values present in the countries column are null. Let us now clean the ```date_added``` and ```duration``` columns.

In [ ]:
date_added_incomplete_entries = filterIncompleteEntries("date_added")
duration_incomplete_entries = filterIncompleteEntries("duration")

Starting with ```date_added```:

In [ ]:
date_added_incomplete_entries

There's only nine entries here, and none of their ```listed_in``` values seem to be correlated. The best course of action is to simply mark the first of January on their ```release_year``` as their ```date_added```.

In [ ]:
null_date_added = netflix["date_added"].isnull()
netflix.loc[null_date_added, "date_added"] = f"January 1, {netflix.loc[null_date_added, 'release_year']}"

We check our work:

In [ ]:
checkForNullEntries()

Now we just need to clean ```duration```.

In [ ]:
duration_incomplete_entries

While all these entries are movies and are from the United States, they make up a small portion of the data and their duration cannot be replaced. This data is best deleted.

In [ ]:
netflix = netflix.drop(netflix[netflix["duration"].isnull()].index)

Let us now check if we have missed any missing data:

In [ ]:
checkForNullEntries()

The final step before we can analyse our data is to sort the table. While not strictly needed, it'll make the task of examining the table itself easier.

In [ ]:
netflix = netflix.sort_values(["country", "release_year", "type", "listed_in"])
netflix

There's still one problem left with our data: the presence of some 